## KMV Analysis of Troubled Stocks and CDS Rating of those stocks



In [18]:
import yfinance as yf
import pandas as pd
from datetime import datetime as dt

company = input('What company would you like to evaluate? Please input the ticker of said company in the prompt.')

def get_data(company):

    ticker = yf.Ticker(company)
    price = ticker.fast_info['previous_close']
    rf_rate = yf.Ticker('^TNX').fast_info['previous_close'] #risk free rate
    market_cap = ticker.fast_info['market_cap'] #market capitalization
    balance_sheet = pd.DataFrame(ticker.balance_sheet) #balance sheet
    total_debt = balance_sheet.loc['Total Debt'].iloc[0] #total debt
    total_assets = balance_sheet.loc['Total Assets'].iloc[0] #total assets

    #Calculating the historical volatility
    df = yf.download(company, period='1y') 
    stock_vol= df['Close'].pct_change().rolling(2).std().mean()

    return company, price, stock_vol, market_cap, total_debt, total_assets, rf_rate

data = get_data(company)
print("--------------------------------------------------------")
print(f"You've chosen the ticker:      {company}")
print("--------------------------------------------------------")
print(f'Last Close:                   ${data[1]}')
print(f"Historical Volatility:         {data[2]*100:.2f}%")
print(f"Market Capitalization:        ${data[3]:.0f}")
print(f"Total Debt:                   ${data[4]:.0f}")
print(f"Total Assets:                 ${data[5]:.0f}")
print(f'Risk Free Rate:                {data[6]:.2f}%')
print(f'--------------------------------------------------------')
print(f'Simply run the next cell to get the Probability of default of your Ticker.')

[*********************100%%**********************]  1 of 1 completed
--------------------------------------------------------
You've chosen the ticker:      TSLA
--------------------------------------------------------
Last Close:                   $190.28
Historical Volatility:         2.36%
Market Capitalization:        $601001745302
Total Debt:                   $9573000000
Total Assets:                 $106618000000
Risk Free Rate:                4.27%
--------------------------------------------------------
Simply run the next cell to get the Probability of default of your Ticker.


In [21]:
from scipy.stats import norm
from scipy.optimize import minimize
import numpy as np
def calcEDF(data):


    stock_vol = data[2]
    stock_val = data[3]
    default_point = data[4]
    assets = data[5]
    rf_rate = data[6]
    T = 1
    
    def Estimate(x):
        asset_val, asset_vol = x

        # Black-Scholes-Merton model
        d1 = (np.log(asset_val/default_point) + (rf_rate + asset_vol**2 / 2) * T) / (asset_vol * np.sqrt(T))
        d2 = d1 - asset_vol * np.sqrt(T)

        #Implementation of the formulas
        equity_val = asset_val*norm.cdf(d1) - np.exp(-rf_rate * T) * default_point * norm.cdf(d2)
        equity_vol = asset_vol*norm.cdf(d1)*asset_val/equity_val
    
        # Residual 1: difference between calculated value and market value of equity
        residual_1 = (stock_val - equity_val)**2 

        # Residual 2: difference between calculated volatility and market volatility
        residual_2 = (stock_vol - equity_vol)**2 
    
        # Objective function combining residuals
        return residual_1 + residual_2

    # Initial guesses
    initial_guess = [assets, stock_vol]

    # Bounds to ensure positive asset value and volatility
    bounds = [(1, None), (0.0001, None)]

    # Optimization
    result = minimize(Estimate, initial_guess, bounds=bounds, method='L-BFGS-B')
    asset_val, asset_vol = result.x

     # Black-Scholes-Merton model
    d1 = (np.log(asset_val*np.exp(rf_rate*T)/default_point) + (rf_rate + asset_vol**2 / 2) * T) / (asset_vol * np.sqrt(T))
    d2 = d1 - asset_vol * np.sqrt(T)

    def ProbabilityofDefault():
        PoD = norm.cdf(-d2)
        return PoD
    
    def ImpliedCreditSpread():
        credit_spread = -1/T * (np.log(1 - ProbabilityofDefault()) + np.log(1+rf_rate))
        return credit_spread
    
    return asset_val, asset_vol, ProbabilityofDefault(), ImpliedCreditSpread()

results = calcEDF(data)

print("--------------------------------------------------------")
print(f"The Estimated Distance to Default for:      {company}")
print("--------------------------------------------------------")
print(f"Estimated Assets Value:                    ${results[0]:.2f}")
print(f"Estimated Assets Volatility:                {results[1]*100:.2f}%")
print(f"Estimated Probability of Default:           {results[2]}%")
print(f"Estimated Implied Credit Spread:           {results[3]*100:.2f}%")


--------------------------------------------------------
The Estimated Distance to Default for:      TSLA
--------------------------------------------------------
Estimated Assets Value:                    $601135990616.06
Estimated Assets Volatility:                2.36%
Estimated Probability of Default:           0.0%
Estimated Implied Credit Spread:           -0.00%


In [10]:
import random 
stocks = pd.DataFrame()

with open('stocks.txt', 'r') as f:
    tickers = []
    tickers = f.readlines()

for i, ticker in enumerate(random.sample(tickers, 10)):
        tickers[i] = ticker[:-1] 
        try:
            data = get_data(tickers[i])
            edf = calcEDF(data)
            s = pd.DataFrame(data=[[f'${data[1]}', f'{data[2]*100:.2f}%', f'${data[3]:.0f}', f'${data[4]:.0f}', f'{edf[0]:.0f}', f'{edf[1]*100:.2f}%', f'{100*edf[2]:.2f}%']], index=[tickers[i]], 
                            columns=['Close', 'Daily Volatility', 'Market_Cap', 'Total_Debt', 'Assets_Value', 'Assets_Volatility', 'Estimated_Default_Probability'])
            stocks = pd.concat([s, stocks], axis=0)
        except:
            print("Company doesn't use standard terms in balance sheet. Current Debt, Long Term Debt and Total Assets couldn't be parsed.")
        
        
stocks

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Close,Daily Volatility,Market_Cap,Total_Debt,Assets_Value,Assets_Volatility,Estimated_Default_Probability
IMKTA,$76.21,0.96%,$1447563207,$595465387,1455514560,0.96%,0.00%
KREF,$9.35,1.59%,$648085029,$6059921000,729004299,1.42%,0.00%
NBHC,$33.46,1.87%,$1275240375,$438890000,1281100947,1.86%,0.00%
CNXN,$65.4,1.24%,$1718156140,$8164000,1718265146,1.24%,0.00%
MODG,$14.8,2.06%,$2786206650,$3567500000,2833844154,2.05%,0.00%
MXL,$19.0,2.46%,$1566431001,$157750000,1568537458,2.46%,0.00%
ETWO,$3.86,2.95%,$1252426942,$1081412000,1266867232,2.92%,0.00%
ACLS,$120.59,2.23%,$3941810950,$46414000,3942430696,2.23%,0.00%
AEIS,$99.97,1.64%,$3815306701,$484493000,3821776220,1.64%,0.00%
ENFN,$8.5,1.89%,$788336855,$6989000,123783000,1.89%,0.00%
